In [1]:
#!pip install google-cloud-storage
import pandas as pd
from google.cloud import storage
# import os

# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "Hedwig/00_Data/msca310019-capstone-49b3-6876cdba6fac.json"

In [14]:
#!pip install pandarallel
import multiprocessing

num_processors = multiprocessing.cpu_count()
print(f'Available CPUs: {num_processors}')

import pandarallel
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=num_processors-1, use_memory_fs=False)

Available CPUs: 8
INFO: Pandarallel will run on 7 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
client = storage.Client()
bucket = client.get_bucket('user-scripts-msca310019-capstone-49b3')

In [3]:
blob = bucket.blob('data/conversation_emails.csv')
content = blob.download_as_string()

In [4]:
import io
df = pd.read_csv(io.BytesIO(content))

In [6]:
df.drop_duplicates(subset=['message'], keep='first',inplace=True)

In [16]:
thread_counts=df.groupby(['thread'], as_index=False)['message'].agg('count')
thread_counts.head(20)

,thread,message
0,0,8432
1,1,2546
2,2,68
3,3,137
4,4,1
5,5,2
6,6,2
7,7,3
8,8,3
9,9,4


In [17]:
thread_multi=thread_counts[thread_counts.message>=2]
thread_multi.drop([0,1], inplace = True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [19]:
multi_threads_list=thread_multi.thread.to_list()

In [21]:
df['flg_multi']=df['thread'].apply(lambda x: 1 if x in multi_threads_list else 0)

In [23]:
df_message_v3=df[df.flg_multi==1]
df_message_v3.head()

,Unnamed: 0,time,subject,thread,sender,recipient,cc,bcc,message,flg_multi
2,2,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",Re: test,2,0,[3],[],[],test successful. way to go!!!,1
4,4,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",Re: Hello,3,0,[5],[],[],Let's shoot for Tuesday at 11:45.,1
5,5,"Thu, 31 Aug 2000 04:17:00 -0700 (PDT)",Re: Hello,3,0,[5],[],[],"Greg, How about either next Tuesday or Thursda...",1
8,8,"Tue, 17 Oct 2000 02:26:00 -0700 (PDT)",Re: High Speed Internet Access,5,0,[9],[],[],1. login: pallen pw: ke9davis I don't think th...,1
9,9,"Mon, 16 Oct 2000 06:44:00 -0700 (PDT)",FW: fixed forward or other Collar floor gas pr...,6,0,[10],[],[],---------------------- Forwarded by Phillip K ...,1


In [24]:
df_message_v3['time'] = pd.to_datetime(df_message_v3['time'])
df_message_v3

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,time,subject,thread,sender,recipient,cc,bcc,message,flg_multi
2,2,2000-10-18 03:00:00-07:00,Re: test,2,0,[3],[],[],test successful. way to go!!!,1
4,4,2000-08-31 05:07:00-07:00,Re: Hello,3,0,[5],[],[],Let's shoot for Tuesday at 11:45.,1
5,5,2000-08-31 04:17:00-07:00,Re: Hello,3,0,[5],[],[],"Greg, How about either next Tuesday or Thursda...",1
8,8,2000-10-17 02:26:00-07:00,Re: High Speed Internet Access,5,0,[9],[],[],1. login: pallen pw: ke9davis I don't think th...,1
9,9,2000-10-16 06:44:00-07:00,FW: fixed forward or other Collar floor gas pr...,6,0,[10],[],[],---------------------- Forwarded by Phillip K ...,1
...,...,...,...,...,...,...,...,...,...,...
245121,517393,2001-11-30 14:53:21-08:00,RE:,208,322,[51654],[],[],"not yet, rob has been in the court house all d...",1
245122,517394,2001-11-29 11:41:51-08:00,FW: Hello,11292,322,[51654],[],[],"-----Original Message----- From: \t""Lerner, L...",1
245123,517395,2001-11-29 11:04:24-08:00,RE:,208,322,[51636],[],[],"very busy, jess is ok just scared -----Origina...",1
245126,517398,2001-11-28 07:20:00-08:00,RE: CONFIDENTIAL,12455,322,[3541],[],[],"2 -----Original Message----- From: \tDoucet, D...",1


In [25]:
df_message_v3.sort_values(by=['thread','time'], ascending=[True,True], inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [56]:
import ast
df_message_v3['recipient'] = df_message_v3['recipient'].apply(lambda x: ast.literal_eval(x))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [90]:
test = df_message_v3[df_message_v3.thread<31]

In [62]:
#test = test.explode('recipient')

In [103]:
df_message_v3.thread.unique()

array([     2,      3,      5, ..., 137844, 137870, 137877])

In [107]:
threads = test.thread.unique()

for thread in threads:
    thread_df = df_message_v3[df_message_v3.thread==thread]
    
    r = list(thread_df.recipient)
    s = list(thread_df.sender)
    
    pairs = []

    for i in range(len(thread_df)):
        for j in range(i):
            if(s[i] in r[j]):
                pairs.append((j,i))

    for pair in pairs:
        message = thread_df.iloc[pair[0]]
        reply = thread_df.iloc[pair[1]]
        reply_new = reply.add_prefix('reply_')
        concatenated_row = pd.concat([message, reply_new], axis=0)
    print(pairs)

[(1, 2), (2, 3), (2, 5), (2, 6), (2, 7), (4, 22), (4, 23), (35, 36), (38, 39), (39, 40), (38, 41), (40, 41), (39, 42), (41, 42), (48, 49), (49, 50), (33, 51), (33, 52), (48, 54), (50, 54), (53, 54), (55, 57), (55, 59), (60, 61), (62, 63), (63, 64), (66, 67)]
[(6, 7), (17, 18), (17, 19), (21, 22), (24, 27), (25, 27), (27, 28), (24, 29), (25, 29), (28, 29), (27, 30), (29, 30), (26, 31), (40, 48), (47, 48), (52, 53), (48, 54), (48, 56), (61, 62), (61, 65), (62, 66), (65, 66), (71, 72), (48, 77), (32, 79), (33, 79), (41, 79), (42, 79), (76, 79), (32, 82), (33, 82), (41, 82), (42, 82), (76, 82), (32, 83), (33, 83), (41, 83), (42, 83), (76, 83), (32, 84), (33, 84), (41, 84), (42, 84), (76, 84), (48, 85), (48, 86), (48, 89), (48, 90), (72, 93), (2, 94), (72, 95), (2, 96), (72, 97), (98, 99), (2, 101), (72, 107), (106, 108), (95, 111), (97, 111), (32, 112), (33, 112), (41, 112), (42, 112), (76, 112), (112, 113), (32, 115), (33, 115), (41, 115), (42, 115), (76, 115), (113, 115), (114, 115), (11

In [106]:
import pandas as pd

# create example DataFrame
data = {'A': [1, 2], 'B': [3, 4], 'C': [5, 6]}
df = pd.DataFrame(data)

# select rows to concatenate
row1 = df.iloc[0]
row2 = df.iloc[1]

# add prefix to columns of row2
row2_prefixed = row2.add_prefix('prefix_')

# concatenate rows
concatenated_row = pd.concat([row1, row2_prefixed], axis=0)

# create new DataFrame with concatenated row
new_df = pd.DataFrame(concatenated_row).T

# reset index of new DataFrame
new_df = new_df.reset_index(drop=True)

# print new DataFrame
print(new_df)

   A  B  C  prefix_A  prefix_B  prefix_C
0  1  3  5         2         4         6


In [105]:
df_message_v3[df_message_v3.thread==8]

,Unnamed: 0,time,subject,thread,sender,recipient,cc,bcc,message,flg_multi
157275,341142,2000-10-03 09:29:00-07:00,Re: 2001 Margin Plan,8,457,[15],[],[],Can send you me a copy of the 2000 plan for th...,1
157268,341134,2000-10-04 11:09:00-07:00,Re: 2001 Margin Plan,8,457,[34988],[],[],---------------------- Forwarded by Scott Neal...,1
15,15,2000-10-05 05:55:00-07:00,Re: 2001 Margin Plan,8,0,[15],[],[],"Paula, 35 million is fine Phillip",1


In [94]:
test.iloc[1]

Unnamed: 0                                               393931
time                                  1999-06-09 04:18:00-07:00
subject                                                RE: test
thread                                                        2
sender                                                     5552
recipient                                               [40034]
cc                                                           []
bcc                                                          []
message       How about this Friday ? Julie has not left yet...
flg_multi                                                     1
Name: 185048, dtype: object